In [17]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [18]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version

Pandas : 1.0.1
Numpy : 1.18.1
Scikit-Learn : 0.22.1
Python 3.7.6


## Data Cleansing & Pre-Processing


In [19]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [20]:
# # 날짜 처리
# data = pd.read_csv('jeju_data_ver1/201901-202003.csv')
# data = data.fillna('')
# data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
# data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
# data = data.drop(['REG_YYMM'], axis=1)

In [21]:
# 날짜 처리
df = pd.read_csv('jeju_data_ver1/201901-202003.csv')
df = df.fillna('')
df['year'] = df['REG_YYMM'].apply(lambda x: grap_year(x))
df['month'] = df['REG_YYMM'].apply(lambda x: grap_month(x))
df = df.drop(['REG_YYMM'], axis=1)

In [22]:
# 데이터 정제

df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
# df['CARD_SIDO_NM'] = df.apply(lambda x: x['CARD_SIDO_NM'] + x['CARD_CCG_NM'], axis=1)
# df = df.drop(['CARD_CCG_NM'], axis=1)
# df['HOM_SIDO_NM'] = df.apply(lambda x: x['HOM_SIDO_NM'] + x['HOM_CCG_NM'], axis=1)
# df = df.drop(['HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [7]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

## 코로나 피쳐
1. is_covid: 2020/03은 1 나머지는 0으로 처리
2. is_covid_plus: 코로나 이후 cnt 상승한 업종들은 1 나머진는 0 처리
    * https://dacon.io/competitions/official/235615/codeshare/1302?page=1&dtype=recent&ptype=pub 참고.
    
    
* 결과: 
    * 기본: 1.27941
    * is_covid: 1.26016 - 제일좋음
    * is_covid_plus: 1.26935
    * is_covid+is_covid_plus: 1.26858

In [8]:
def is_covid(x):
    if x['year'] == 2020:
        if x['month'] ==3:
            return 1
    else:
        return 0

def is_covid_plus(x):
    # ['건강보조식품 소매업', '골프장 운영업','과실 및 채소 소매업', '그외 기타 종합 소매업', '기타 대형 종합 소매업', '기타음식료품위주종합소매업', '내항 여객 운송업', '빵 및 과자류 소매업', '수산물 소매업', '슈퍼마켓', '육류 소매업']
    # [ 0,  1,  2,  6,  7, 11, 12, 17, 19, 20, 25])    
    if x['STD_CLSS_NM'] in [ 0,  1,  2,  6,  7, 11, 12, 17, 19, 20, 25]:
        return 1
    else:
        return 0   
    
df_num['covid'] = df_num.apply(lambda x: is_covid(x), axis=1)
df_num['is_covid_plus'] = df_num.apply(lambda x: is_covid_plus(x), axis=1)

## Feature Engineering & Initial Modeling


In [10]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [11]:
k = int(len(x)*0.9)

In [12]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

## Model Tuning & Evaluation


In [13]:
import lightgbm as lgb


In [14]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [15]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [16]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.9767
[200]	valid_0's rmse: 1.79663
[300]	valid_0's rmse: 1.69108
[400]	valid_0's rmse: 1.59567
[500]	valid_0's rmse: 1.51956
[600]	valid_0's rmse: 1.44948
[700]	valid_0's rmse: 1.4054
[800]	valid_0's rmse: 1.3533
[900]	valid_0's rmse: 1.30941
[1000]	valid_0's rmse: 1.26858
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.26858


## Conclusion & Discussion


In [16]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [17]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [18]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [20]:
# 제출 파일 만들기
submission = pd.read_csv('jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,2.353337e+08
1,202004,강원,골프장 운영업,1.601208e+09
2,202004,강원,과실 및 채소 소매업,6.451061e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,6.329203e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.724102e+07


In [45]:
submission.shape

(1394, 4)